In [1]:
%load_ext autoreload
%autoreload 2

import molpy as mp
# import molvis as mv

class Methane(mp.Struct):

    def __init__(self):
        super().__init__()
        C = self.add_atom(name="C", xyz=(0.1, 0, -0.07))
        H1 = self.add_atom(name="H1", xyz=(-0.1, 0, -0.07))
        H2 = self.add_atom(name="H2", xyz=(0., 0.1, 0.07))
        H3 = self.add_atom(name="H3", xyz=(0., -0.1, 0.07))
        self.add_bond(C, H1)
        self.add_bond(C, H2)
        self.add_bond(C, H3)

methane = Methane()
# mp.io.write_pdb("methane.pdb", methane)

In [2]:
ch3 = mp.builder.CH3()
ch3.translate(-ch3["atoms"][0].xyz)

class Ethane(mp.Struct):

    def __init__(self):
        super().__init__()
        ch3_1 = mp.builder.CH3()
        ch3_1.translate(-ch3_1["atoms"][0].xyz)
        ch3_2 = mp.builder.CH3()
        ch3_2.translate(-ch3_2["atoms"][0].xyz)
        ch3_2.rotate(180, axis=(0, 1, 0))
        self.add_struct(ch3_1)
        self.add_struct(ch3_2)
        self.add_bond(ch3_1["atoms"][0], ch3_2["atoms"][0])

ethane = Ethane()
# Get topology
topology = ethane.get_topology()

# Calculate angles and dihedrals
angles = ethane.calc_angles(topology)
dihedrals = ethane.calc_dihedrals(topology)
len(ethane["atoms"]) == 8
len(ethane["bonds"]) == 7
len(ethane["angles"]) == 12
len(ethane["dihedrals"]) == 9

False

In [ ]:
ff = mp.ForceField("test")
atomstyle = ff.def_atomstyle("full")
C_type = atomstyle.def_type("C", charge=-0.1)
H_type = atomstyle.def_type("H", charge=0.1)
bondstyle = ff.def_bondstyle("harmonic")
bondstyle.def_type(C_type, C_type, 1.5, 100)
bondstyle.def_type(C_type, H_type, 1.1, 100)
anglestyle = ff.def_anglestyle("harmonic")
anglestyle.def_type(C_type, C_type, H_type, 109.5, 100)
anglestyle.def_type(H_type, C_type, H_type, 109.5, 100)
dihedralstyle = ff.def_dihedralstyle("harmonic")
dihedralstyle.def_type(H_type, C_type, C_type, H_type, 0, 100)


In [11]:
angles

[<Angle: C-C-H1>,
 <Angle: C-C-H2>,
 <Angle: C-C-H3>,
 <Angle: H1-C-H2>,
 <Angle: H1-C-H3>,
 <Angle: H1-C-C>,
 <Angle: H2-C-H3>,
 <Angle: H2-C-C>,
 <Angle: H3-C-C>,
 <Angle: H1-C-H2>,
 <Angle: H1-C-H3>,
 <Angle: H2-C-H3>]

In [4]:
len(ethane["dihedrals"])

0

In [5]:
len(ethane["atoms"])

8

In [6]:
len(ethane["bonds"])

7